# QSVM

In [28]:
import pandas as pd
from qiskit import Aer
from qiskit.algorithms.state_fidelities import ComputeUncompute
from qiskit.primitives import Sampler
from qiskit.utils import algorithm_globals, QuantumInstance
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.kernels import QuantumKernel, FidelityQuantumKernel
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

In [29]:
df = pd.read_csv('datasets/adhoc.csv', index_col=0)
df

,feature_1,feature_2,label
0,0.376991,1.445133,0.0
1,2.450442,3.267256,0.0
2,3.455752,3.078761,0.0
3,5.592035,1.130973,0.0
4,4.272566,2.576106,0.0
...,...,...,...
9995,5.654867,2.073451,1.0
9996,3.644247,0.251327,1.0
9997,6.157522,5.592035,1.0
9998,2.450442,0.691150,1.0


In [30]:
feature_dimension = df.shape[1] - 1
print(f"Feature dimension: {feature_dimension}")

Feature dimension: 2


In [31]:
# Split the data into training and test sets
all_features = df.copy()
all_labels = all_features.pop('label')

train = df.sample(frac=0.75, random_state=42)
test = df.drop(train.index)

# Separate the features from the labels
train_features = train.copy()
test_features = test.copy()

train_labels = train_features.pop('label')
test_labels = test_features.pop('label')

In [32]:
algorithm_globals.random_seed = 42
quantum_instance = QuantumInstance(Aer.get_backend("aer_simulator"), shots=1024)

In [33]:
feature_map = ZZFeatureMap(feature_dimension=feature_dimension, reps=2, entanglement='linear')
kernel = QuantumKernel(feature_map=feature_map, quantum_instance=quantum_instance)
# kernel = FidelityQuantumKernel(fidelity=ComputeUncompute(sampler=Sampler()))
svc = SVC(kernel=kernel.evaluate)

In [34]:
svc.fit(train_features, train_labels)
score = svc.score(test_features, test_labels)
print(f"QSVM score: {score}")

/Users/lukas/miniforge3/envs/qc/lib/python3.9/site-packages/numpy/linalg/linalg.py:2146: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/lukas/miniforge3/envs/qc/lib/python3.9/site-packages/numpy/linalg/linalg.py:2146: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


KeyboardInterrupt: 

In [19]:
scores = cross_val_score(svc, all_features, all_labels, cv=10)
print(scores)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

/Users/lukas/miniforge3/envs/qc/lib/python3.9/site-packages/numpy/linalg/linalg.py:2146: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/lukas/miniforge3/envs/qc/lib/python3.9/site-packages/numpy/linalg/linalg.py:2146: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/lukas/miniforge3/envs/qc/lib/python3.9/site-packages/numpy/linalg/linalg.py:2146: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/lukas/miniforge3/envs/qc/lib/python3.9/site-packages/numpy/linalg/linalg.py:2146: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/lukas/miniforge3/envs/qc/lib/python3.9/site-packages/numpy/linalg/linalg.py:2146: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/lukas/miniforge3/envs/qc/lib/python3.9/site-packages/numpy/l

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
1.00 accuracy with a standard deviation of 0.00


In [35]:
classical_svc = SVC()
classical_svc.fit(train_features, train_labels)
score = classical_svc.score(test_features, test_labels)

In [36]:
print(f"Classical SVC score: {score}")

Classical SVC score: 0.6756


In [37]:
scores = cross_val_score(classical_svc, all_features, all_labels, cv=10)
scores

array([0.681, 0.674, 0.689, 0.669, 0.692, 0.7  , 0.656, 0.683, 0.686,
       0.683])

In [38]:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))


0.68 accuracy with a standard deviation of 0.01
